# Leakage coefficient fit

> *This notebook estracts the leakage coefficient from the set of 5 us-ALEX smFRET measurements.*

## What it does?

For each measurement, we fit the donor-only peak position of the uncorrected proximity ratio histogram. These values are saved in a `.txt` file. This notebook just performs a weighted mean where the weights are the number of bursts in each measurement.

This notebook read data from the file:

In [ ]:
#bsearch_ph_sel = 'all-ph'
#bsearch_ph_sel = 'Dex'
bsearch_ph_sel = 'DexDem'

data_file = 'results/usALEX-5samples-PR-raw-%s.csv' % bsearch_ph_sel

To recompute the PR data used by this notebook run the 
[8-spots paper analysis](8-spots paper analysis.ipynb) notebook.

# Computation

In [ ]:
from __future__ import division
import numpy as np
import pandas as pd
from IPython.display import display
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
%config InlineBackend.figure_format='retina'  # for hi-dpi displays

sns.set_style('whitegrid')
palette = ('Paired', 10)
sns.palplot(sns.color_palette(*palette))
sns.set_palette(*palette)

In [ ]:
data = pd.read_csv(data_file).set_index('sample')
data

In [ ]:
display(data[['E_pr_do_gauss', 'E_pr_do_kde', 'E_pr_do_hsm', 'n_bursts_do']])
print('KDE Mean (%):     ', data.E_pr_do_kde.mean()*100)
print('KDE Std. Dev. (%):', data.E_pr_do_kde.std()*100)

In [ ]:
d = data[['E_pr_do_gauss', 'E_pr_do_kde', 'E_pr_do_hsm']]#, 'n_bursts_do']]
d.plot(lw=3);

## Create Leakage Table

In [ ]:
E_table = data[['E_pr_do_gauss', 'E_pr_do_kde']]
E_table

In [ ]:
lk_table = E_table / (1 - E_table)
lk_table.columns = [c.replace('E_pr_do', 'lk') for c in E_table.columns]
lk_table['num_bursts'] = data['n_bursts_do']
lk_table

## Average leakage coefficient

In [ ]:
data.E_pr_do_kde

In [ ]:
lk_table.lk_kde

In [ ]:
E_m = np.average(data.E_pr_do_kde, weights=data.n_bursts_do)
E_m

In [ ]:
k_E_m = E_m / (1 - E_m)
k_E_m

In [ ]:
k_m = np.average(lk_table.lk_kde, weights=data.n_bursts_do)
k_m

# Conclusions

Either averaging $E_{PR}$ or the corresponding $k = n_d/n_a$ the result for the leakage coefficient is ~10 % (D-only peak fitted finding the maximum of the KDE).

# Save data

## Full table

In [ ]:
stats = pd.concat([lk_table.mean(), lk_table.std()], axis=1, keys=['mean', 'std']).T
stats

In [ ]:
table_to_save = lk_table.append(stats)
table_to_save = table_to_save.round({'lk_gauss': 5, 'lk_kde': 5, 'num_bursts': 2})
table_to_save

In [ ]:
table_to_save.to_csv('results/table_usalex_5samples_leakage_coeff.csv')

## Average coefficient

In [ ]:
'%.5f' % k_m

In [ ]:
with open('results/usALEX - leakage coefficient %s.csv' % bsearch_ph_sel, 'w') as f:
    f.write('%.5f' % k_m)